In [1]:
import sys
sys.path.append("../")

In [2]:
from __future__ import division
from full_prod_DRA import *
from buchi import buchi_from_ltl
import numpy as np
from env_sensing_error_small import *
import scipy
# from plot_path_for_prod import *
from graphviz import Source
from qnetwork import *
from utils import *
import matplotlib.pyplot as plt
from Plot_Path import *
import tensorflow as tf
from dra_planning import *
import time
from dra_planning import convert_path_to_action

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B"
# LTL = "[] (p1 -> !(X p1) U (p2 || p3) ) && []<>p1"
# LTL = "T && []<>A && []<>B"
# LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C && []!D"
LTL = "<>(A && <>(B && <> T)) && []<>A && []<>B && []!C"
# LTL = "<>(A && <>(B && <> T))"
# LTL = "<>(A && <>B) && <>[]T && []!C"
# LTL = "<>(A && <>T) && []!C"
# LTL = "<>(A && <>(B && <>T)) && []<>(A||T) && []<>B && []!C"
# LTL = "<>(A && <>(B && <>T)) && []!C"
# LTL = "<>(A && <>D) && <>(B && <>E) && []<>T && []<>(D || E) && []!C"

LEARNING_RATE = 0.0015
GAMMA = 0.99
# GAMMA = 0.7
TAU = 0.001
BUFFER_SIZE = 10**6
MINIBATCH_SIZE = 64
RANDOM_SEED = 191
MAX_EPISODES = 50000
MAX_EPISODE_LEN = 1000
file_appendix = "Pure_Imitation_Large_" + time.ctime()[4:16].replace("  ","").replace(" ","_").replace(":","-") + LTL
# file_appendix = "Feb8_03-47"
SUMMARY_DIR = './results/tf_ddqn_' + file_appendix
SAVE_DIR = "./saved_model/" + file_appendix + "/ddqn.ckpt"
EXPLORATION_RATE = 0.7
LR_DECAY_TRUNCATION = -800
RESTORE = 0
if sys.platform == "darwin":
    DEVICE = "/device:CPU:0"
else:
    DEVICE = "/device:GPU:0"

In [4]:
env = CurrentWorld(LTL)
prod_planner = Prod_Planning(env, LTL)
# with open("my.dot", "r") as dotfile:
#     text = dotfile.read()
# Source(text)

In [5]:
guide_policy = defaultdict(lambda: [])
prod_planner.get_global_opt()
prod_planner.get_opt_rabin()

In [6]:
for i in range(10):
    for rabin_s in env.rabin.states:
        for s_x in range(env.shape[0]):
            for s_y in range(env.shape[1]):
                env.step(np.random.randint(env.nA))
                prod_planner.update_wfts_ap()
                local_plan = prod_planner.get_local_opt((s_x,s_y), prod_planner.get_next_ltl(rabin_s))
                if local_plan == None:
                    guide_policy[(s_x,s_y,rabin_s)] += [np.random.randint(env.nA)]
                else:
                    guide_policy[(s_x,s_y,rabin_s)] += [convert_path_to_action(local_plan)]

In [7]:
for i in guide_policy.keys():
    guide_policy[i] = scipy.stats.mode(guide_policy[i])[0][0]

In [8]:
replay_buffer = ReplayBuffer(BUFFER_SIZE, RANDOM_SEED)

In [9]:
for epi in range(100):

    s = env.reset()
    s = np.unravel_index(s, env.shape)

    for t in range(50):
        a = guide_policy[s]
        s2, r, terminal, info = env.step(a)
        s2 = np.unravel_index(s2, env.shape)
        replay_buffer.add(np.reshape(s, (3,)), np.reshape(a, (1,)), r,
                              terminal, np.reshape(s2, (3,)))
        s = s2

In [10]:
def train(sess, env, qnet, replay_buffer):
    
    global EXPLORATION_RATE
  
    summary_ops, summary_vars = build_summaries()

    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(SUMMARY_DIR, sess.graph)
    
    qnet.update_target()
    
#     replay_buffer = ReplayBuffer(BUFFER_SIZE, RANDOM_SEED)
    
    # Initialize Buffer with guide_policy
    
    # Start Training
    
    for num_epi in range(MAX_EPISODES):

        s = env.reset()
        s = [list(np.unravel_index(s, env.shape))]

        ep_reward = 0
        ep_ave_max_q = 0
        
        reward_list = []

        for j in range(MAX_EPISODE_LEN):

            a = np.argmax(qnet.predict_q(np.reshape(s, (1, qnet.state_dim))))
    
            if np.random.rand(1) < EXPLORATION_RATE:
                a = np.random.randint(0,qnet.action_dim)
                s2, r, terminal, info = env.step(a)
            else:
                s2, r, terminal, info = env.step(a)
            
            s2 = list(np.unravel_index(s2, env.shape))

            replay_buffer.add(np.reshape(s, (qnet.state_dim,)), np.reshape(a, (1,)), r,
                              terminal, np.reshape(s2, (qnet.state_dim,)))

            # Keep adding experience to the memory until
            # there are at least minibatch size samples
            if replay_buffer.size() > MINIBATCH_SIZE:
                s_batch, a_batch, r_batch, t_batch, s2_batch = replay_buffer.sample_batch(MINIBATCH_SIZE)

                # Calculate targets
                target_q = qnet.predect_target(s2_batch)

                y_i = []
                for k in range(MINIBATCH_SIZE):
                    if t_batch[k]:
                        y_i.append(r_batch[k])
                    else:
                        y_i.append(r_batch[k] + GAMMA * np.amax(target_q[k]))

                # Update the critic given the targets
                predicted_q_value, _ = qnet.train(s_batch, a_batch, np.reshape(y_i, (MINIBATCH_SIZE, 1)), num_epi)

                ep_ave_max_q += np.amax(predicted_q_value)
                
                # Update target networks
                qnet.update_target()

            s = s2
            ep_reward += r

            if terminal or j == MAX_EPISODE_LEN-1:
                
                if EXPLORATION_RATE > 0.05 and terminal:
                    EXPLORATION_RATE = EXPLORATION_RATE*0.98
                    
                reward_list += [ep_reward]
                
                if np.average(reward_list[-10:]) > LR_DECAY_TRUNCATION:
                    qnet.decay_learning_rate(0.98)

#                 summary_str = sess.run(summary_ops, feed_dict={
#                     summary_vars[0]: ep_reward,
#                     summary_vars[1]: ep_ave_max_q / float(j),
#                     summary_vars[2]: EXPLORATION_RATE,
#                     summary_vars[3]: qnet.get_learning_rate()                    
#                 })

#                 writer.add_summary(summary_str, num_epi)
#                 writer.flush()

                print('| Reward: {:d} | Episode: {:d} | Qmax: {:.4f} | Exploration: {:.6f} | Step: {:d} | LR: {:.8f}'.format(int(ep_reward), \
                        num_epi, (ep_ave_max_q / float(j)), EXPLORATION_RATE, j, qnet.get_learning_rate()))
                
                f = open("stats/" + file_appendix + "_stats.txt", "ab")
                f.write("| Reward: " + str(int(ep_reward)) 
                        +" | Episode: " + str(num_epi) 
                        + " | Qmax: " + str(ep_ave_max_q / float(j)) 
                        + " | Exploration: " + str(EXPLORATION_RATE)
                        + " | Step: " + str(j)
                        + " | LR:" + str(qnet.get_learning_rate()) + "\n")
                f.close()
 
                
                break
    

In [11]:
config=tf.ConfigProto(log_device_placement=False)
config.gpu_options.allow_growth = True
with tf.Session(config=config) as sess:
       
    state_dim = 3
    action_dim = 5
    
    if RESTORE:
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)
        Qnet.saver.restore(sess, RESTORE_PATH)
        train(sess, env, Qnet, replay_buffer)
        
    else:
        np.random.seed(RANDOM_SEED)
        tf.set_random_seed(RANDOM_SEED)
        env.seed(RANDOM_SEED)
    
        Qnet = QNet(sess, state_dim, action_dim, LEARNING_RATE, TAU, MINIBATCH_SIZE, SAVE_DIR, DEVICE)

        train(sess, env, Qnet, replay_buffer)

/home/ubuntu/anaconda3/envs/tensorflow_p27/lib/python2.7/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


DDQN Saved
| Reward: -10471 | Episode: 0 | Qmax: -4.1526 | Exploration: 0.686000 | Step: 905 | LR: 0.00150000
| Reward: -13780 | Episode: 1 | Qmax: -16.9925 | Exploration: 0.686000 | Step: 999 | LR: 0.00150000
| Reward: -12340 | Episode: 2 | Qmax: -22.2568 | Exploration: 0.686000 | Step: 999 | LR: 0.00150000
| Reward: -12880 | Episode: 3 | Qmax: -26.1182 | Exploration: 0.686000 | Step: 999 | LR: 0.00150000
| Reward: -11260 | Episode: 4 | Qmax: -27.9544 | Exploration: 0.686000 | Step: 999 | LR: 0.00150000
| Reward: -8677 | Episode: 5 | Qmax: -29.6805 | Exploration: 0.672280 | Step: 947 | LR: 0.00150000
| Reward: -14230 | Episode: 6 | Qmax: -30.2839 | Exploration: 0.672280 | Step: 999 | LR: 0.00150000
| Reward: -13780 | Episode: 7 | Qmax: -30.9618 | Exploration: 0.672280 | Step: 999 | LR: 0.00150000
| Reward: -8388 | Episode: 8 | Qmax: -32.2436 | Exploration: 0.658834 | Step: 730 | LR: 0.00150000
| Reward: -11197 | Episode: 9 | Qmax: -32.5946 | Exploration: 0.658834 | Step: 999 | LR: 0.0

| Reward: -2604 | Episode: 82 | Qmax: 48.6348 | Exploration: 0.311990 | Step: 634 | LR: 0.00150000
| Reward: -1856 | Episode: 83 | Qmax: 48.1350 | Exploration: 0.305750 | Step: 327 | LR: 0.00150000
| Reward: -3394 | Episode: 84 | Qmax: 51.3040 | Exploration: 0.305750 | Step: 999 | LR: 0.00150000
| Reward: -1869 | Episode: 85 | Qmax: 51.6178 | Exploration: 0.299635 | Step: 538 | LR: 0.00150000
| Reward: -3486 | Episode: 86 | Qmax: 54.8764 | Exploration: 0.293643 | Step: 931 | LR: 0.00150000
| Reward: -1489 | Episode: 87 | Qmax: 53.4961 | Exploration: 0.287770 | Step: 365 | LR: 0.00150000
| Reward: -672 | Episode: 88 | Qmax: 58.0450 | Exploration: 0.282015 | Step: 142 | LR: 0.00147000
| Reward: -1225 | Episode: 89 | Qmax: 55.7740 | Exploration: 0.276374 | Step: 308 | LR: 0.00147000
| Reward: -2226 | Episode: 90 | Qmax: 57.2959 | Exploration: 0.270847 | Step: 535 | LR: 0.00147000
| Reward: -1823 | Episode: 91 | Qmax: 57.4823 | Exploration: 0.265430 | Step: 564 | LR: 0.00147000
| Reward: -

| Reward: -292 | Episode: 165 | Qmax: 91.6048 | Exploration: 0.059522 | Step: 95 | LR: 0.00044633
| Reward: -270 | Episode: 166 | Qmax: 92.8437 | Exploration: 0.058332 | Step: 91 | LR: 0.00043740
| Reward: -519 | Episode: 167 | Qmax: 92.3584 | Exploration: 0.057165 | Step: 97 | LR: 0.00042866
| Reward: -278 | Episode: 168 | Qmax: 93.3414 | Exploration: 0.056022 | Step: 72 | LR: 0.00042008
| Reward: -227 | Episode: 169 | Qmax: 95.4900 | Exploration: 0.054901 | Step: 57 | LR: 0.00041168
| Reward: -259 | Episode: 170 | Qmax: 92.4638 | Exploration: 0.053803 | Step: 71 | LR: 0.00040345
| Reward: -287 | Episode: 171 | Qmax: 92.5485 | Exploration: 0.052727 | Step: 90 | LR: 0.00039538
| Reward: -336 | Episode: 172 | Qmax: 93.3410 | Exploration: 0.051673 | Step: 67 | LR: 0.00038747
| Reward: -263 | Episode: 173 | Qmax: 92.9848 | Exploration: 0.050639 | Step: 57 | LR: 0.00037972
| Reward: -344 | Episode: 174 | Qmax: 92.8339 | Exploration: 0.049627 | Step: 120 | LR: 0.00037213
| Reward: -297 | Ep

| Reward: -369 | Episode: 248 | Qmax: 99.5659 | Exploration: 0.049627 | Step: 91 | LR: 0.00008866
| Reward: -324 | Episode: 249 | Qmax: 99.1125 | Exploration: 0.049627 | Step: 154 | LR: 0.00008689
| Reward: -295 | Episode: 250 | Qmax: 99.5777 | Exploration: 0.049627 | Step: 107 | LR: 0.00008515
| Reward: -338 | Episode: 251 | Qmax: 99.0740 | Exploration: 0.049627 | Step: 150 | LR: 0.00008345
| Reward: -386 | Episode: 252 | Qmax: 99.6660 | Exploration: 0.049627 | Step: 90 | LR: 0.00008178
| Reward: -388 | Episode: 253 | Qmax: 98.9004 | Exploration: 0.049627 | Step: 110 | LR: 0.00008014
| Reward: -531 | Episode: 254 | Qmax: 99.0009 | Exploration: 0.049627 | Step: 154 | LR: 0.00007854
| Reward: -1589 | Episode: 255 | Qmax: 98.6838 | Exploration: 0.049627 | Step: 762 | LR: 0.00007854
| Reward: -422 | Episode: 256 | Qmax: 98.3718 | Exploration: 0.049627 | Step: 243 | LR: 0.00007697
| Reward: -475 | Episode: 257 | Qmax: 98.4941 | Exploration: 0.049627 | Step: 278 | LR: 0.00007543
| Reward: -

...

| Reward: -356 | Episode: 15997 | Qmax: 101.0352 | Exploration: 0.049627 | Step: 60 | LR: 0.00000000
| Reward: -226 | Episode: 15998 | Qmax: 101.3603 | Exploration: 0.049627 | Step: 56 | LR: 0.00000000
| Reward: -245 | Episode: 15999 | Qmax: 100.5283 | Exploration: 0.049627 | Step: 57 | LR: 0.00000000
DDQN Saved
| Reward: -263 | Episode: 16000 | Qmax: 101.0001 | Exploration: 0.049627 | Step: 57 | LR: 0.00000000
| Reward: -293 | Episode: 16001 | Qmax: 101.2083 | Exploration: 0.049627 | Step: 60 | LR: 0.00000000
| Reward: -223 | Episode: 16002 | Qmax: 101.2202 | Exploration: 0.049627 | Step: 53 | LR: 0.00000000
| Reward: -244 | Episode: 16003 | Qmax: 100.9685 | Exploration: 0.049627 | Step: 56 | LR: 0.00000000
| Reward: -345 | Episode: 16004 | Qmax: 100.8847 | Exploration: 0.049627 | Step: 58 | LR: 0.00000000
| Reward: -226 | Episode: 16005 | Qmax: 100.9379 | Exploration: 0.049627 | Step: 56 | LR: 0.00000000
| Reward: -223 | Episode: 16006 | Qmax: 101.0888 | Exploration: 0.049627 | Step:

| Reward: -356 | Episode: 16078 | Qmax: 100.9978 | Exploration: 0.049627 | Step: 60 | LR: 0.00000000
| Reward: -254 | Episode: 16079 | Qmax: 100.8747 | Exploration: 0.049627 | Step: 57 | LR: 0.00000000
DDQN Saved
| Reward: -343 | Episode: 16080 | Qmax: 100.4648 | Exploration: 0.049627 | Step: 56 | LR: 0.00000000
| Reward: -253 | Episode: 16081 | Qmax: 100.8653 | Exploration: 0.049627 | Step: 56 | LR: 0.00000000
| Reward: -224 | Episode: 16082 | Qmax: 100.7955 | Exploration: 0.049627 | Step: 54 | LR: 0.00000000
| Reward: -225 | Episode: 16083 | Qmax: 101.1135 | Exploration: 0.049627 | Step: 55 | LR: 0.00000000
| Reward: -223 | Episode: 16084 | Qmax: 100.8973 | Exploration: 0.049627 | Step: 53 | LR: 0.00000000
| Reward: -235 | Episode: 16085 | Qmax: 101.0548 | Exploration: 0.049627 | Step: 56 | LR: 0.00000000
| Reward: -324 | Episode: 16086 | Qmax: 100.7036 | Exploration: 0.049627 | Step: 55 | LR: 0.00000000
| Reward: -263 | Episode: 16087 | Qmax: 101.2503 | Exploration: 0.049627 | Step:

| Reward: -224 | Episode: 16159 | Qmax: 101.2320 | Exploration: 0.049627 | Step: 54 | LR: 0.00000000
DDQN Saved
| Reward: -335 | Episode: 16160 | Qmax: 100.7890 | Exploration: 0.049627 | Step: 57 | LR: 0.00000000
| Reward: -248 | Episode: 16161 | Qmax: 100.9877 | Exploration: 0.049627 | Step: 60 | LR: 0.00000000
| Reward: -233 | Episode: 16162 | Qmax: 101.0926 | Exploration: 0.049627 | Step: 54 | LR: 0.00000000
| Reward: -235 | Episode: 16163 | Qmax: 101.0002 | Exploration: 0.049627 | Step: 56 | LR: 0.00000000
| Reward: -245 | Episode: 16164 | Qmax: 100.9202 | Exploration: 0.049627 | Step: 57 | LR: 0.00000000
| Reward: -223 | Episode: 16165 | Qmax: 100.9899 | Exploration: 0.049627 | Step: 53 | LR: 0.00000000
| Reward: -314 | Episode: 16166 | Qmax: 100.8006 | Exploration: 0.049627 | Step: 54 | LR: 0.00000000
| Reward: -225 | Episode: 16167 | Qmax: 101.1132 | Exploration: 0.049627 | Step: 55 | LR: 0.00000000
| Reward: -253 | Episode: 16168 | Qmax: 101.1551 | Exploration: 0.049627 | Step:

DDQN Saved
| Reward: -332 | Episode: 16240 | Qmax: 100.5804 | Exploration: 0.049627 | Step: 54 | LR: 0.00000000
| Reward: -236 | Episode: 16241 | Qmax: 100.8973 | Exploration: 0.049627 | Step: 57 | LR: 0.00000000
| Reward: -255 | Episode: 16242 | Qmax: 100.8244 | Exploration: 0.049627 | Step: 58 | LR: 0.00000000
| Reward: -273 | Episode: 16243 | Qmax: 101.2018 | Exploration: 0.049627 | Step: 58 | LR: 0.00000000
| Reward: -251 | Episode: 16244 | Qmax: 100.7017 | Exploration: 0.049627 | Step: 63 | LR: 0.00000000
| Reward: -246 | Episode: 16245 | Qmax: 100.9784 | Exploration: 0.049627 | Step: 58 | LR: 0.00000000
| Reward: -247 | Episode: 16246 | Qmax: 100.7370 | Exploration: 0.049627 | Step: 59 | LR: 0.00000000
| Reward: -234 | Episode: 16247 | Qmax: 101.1951 | Exploration: 0.049627 | Step: 55 | LR: 0.00000000
| Reward: -333 | Episode: 16248 | Qmax: 101.2368 | Exploration: 0.049627 | Step: 55 | LR: 0.00000000
| Reward: -243 | Episode: 16249 | Qmax: 100.8929 | Exploration: 0.049627 | Step:

KeyboardInterrupt: 